# Sample Foam Plant Planning

## Problem Statement
- Foam plants have complex scheduling process
- Mould arrangement on foram production line (racetracks) have numerous constraints which represent a large number of possible solutions

#### Products and Tools/Molds
- For this POC, we are considering 3 products: __Prod1, Prod2 and Prod3__
- Prod1, Prod2 and Prod3 can be made by tools/molds __Tool1, Tool2, Tool3__
- The number of tools available: __Tool1=10, Tool2=10, Tool3=10__

#### Customer Demand
- Customer demand is provided for 12 weeks which is then udpated weekly
- For this POC, we are considering it for 1 week

| Prod | Week1 |
| --- | --- |
| Prod1 | 5000 |
| Prod2 | 6000 |
| Prod3 | 3000 |

#### Carrier
- For this POC, we are considering that any Tool can be placed on any Carrier.
- One tool can be placed on one carrier.
- The complexity of FBE will be introduced at a later version.

#### Racetrack/Production Line
- Carriers are fixed to the racetrack and production lines which go around at a specific speed. For this POC, we are considering that there is 1 racetrack
- Linespeed of a Ractrack how many carriers finish the production process in a given time. For eg if it is 42 carriers per hour, it would mean 42 products were produced in an hour.
- In this POC, the speed is considered ad __42 carriers per hour__

#### Inventory and Minimum Stock
- For this POC, we will assume there is a begining inventory for this process
- Additionally, we will want that the end of the week, there should be a minimum stock for each product. This could be a derivative of the customer demand, for this POC we are defining it as an absolute value

| Prod | StartingInventory | EndingInventory |
| --- | --- | --- |
| Prod1 | 10 | 50 |
| Prod2 | 20 | 40 |
| Prod3 | 10 | 30 |

#### Factory Schedule
- Factory runs 7 days a week, 3 shifts a day of 8 hours each
- __Breaks in eah shift:__ 10 min, 20 min and 10 min
- __Production Time in each shift__ = 480 mins - 40 mins = 440 mins

- Each shift has 8 hours = __480 mins__
    
    - __Timeslot 1: 140 mins__ = 2 hours, 20 mins
    - __Break  1:  10 mins__
    
    - __Timeslot 2: 160 mins__ = 2 hours, 40 mins
    - __Break  2:  20 mins__
    
    - __Timeslot 3: 140 mins__ = 2 hours, 20 mins
    - __Break 3:   10 mins__

### Modeling

#### __Parameters__

- L = Line speed = 42 carriers per 60 mins
- R = Number of carriers on the racetrack
- D(i) = Weekly demand for part made with Tool i
- B(i) = Begining inventory for part made with Tool i
- I(i) = Ending inventory for part made with Tool i
- Q(i) = Available quantity of Tool i
- A(k) = Total time available in time slot k

__Decisision Variables__

- x(i,j,k) = 1, if Tool i is placed on Carrier j in time slot k, otherwise 0

- y(j,k) = 1, if Changeover happens at the end of Time slot k on carrier j. __This is the changeover__

#### Constraints
- The number of tools placed on the carriers is less than or equal to the available tools i in the time slot k

        Σj x(i,j,k) <= Q(i)

- The number of tools on a carrier is less than or equal to 1


        Σi x(i,j,k) <= 1

- For each product (tool), the production should be greater than or equal to demand plus ending inventory

        B(i) + L * Σk [A(k) * Σj x(i,j,k)] >= D(i) + I(i)

- The changeover value is 1 only when tool i on the carrier j changes between two timeslots. __This is the changeover__

        y[j,k] == 1, if x[i,j,k] != x[i,j,k+1] for any of the tools on a carrier. 
        
        This can be represented as below:
        
        y[j,k] = 1, if Σi Abs(x[i,j,k] - x[i,j,k+1]) >= 1
        y[j,k] = 0, if Σi Abs(x[i,j,k] - x[i,j,k+1]) <= 0

#### Objective
- Minimize the changeover
- Optimize the schedule

### Implementation

In [1]:
import numpy as np
import pandas as pd
import math

import gurobipy as gp
from gurobipy import GRB
from gurobipy import *

In [2]:
# Sets and Indices

# The tools are mapped directly to products
T = ['Tool1', 'Tool2', 'Tool3', 'Tool4']

# Available quantity of Tools
Q = {'Tool1': 10, 'Tool2': 10, 'Tool3': 10, 'Tool4': 10}

# Demand
D = {'Tool1': 8000, 'Tool2': 8000, 'Tool3': 8000, 'Tool4': 7000}

# Starting inventory
B = {'Tool1': 0, 'Tool2': 0, 'Tool3': 0, 'Tool4': 0}

# Ending inventory 
I = {'Tool1': 50, 'Tool2': 40, 'Tool3': 30, 'Tool4': 20}

# Line speed per minute molds produced per minute
L = 42/60

# Carriers
R = ['C1', 'C2', 'C3', 'C4', 'C5']

In [3]:
# Timeslots
# This is for a period of 7 days
# Each day has 3 shifts
# Each shift has 3 time slots or periods as in this case
K = ['D1S1P1', 'D1S1P2', 'D1S1P3', 'D1S2P1','D1S2P2','D1S2P3','D1S3P1','D1S3P2','D1S3P3',
     'D2S1P1', 'D2S1P2', 'D2S1P3', 'D2S2P1','D2S2P2','D2S2P3','D2S3P1','D2S3P2','D2S3P3',
     'D3S1P1', 'D3S1P2', 'D3S1P3', 'D3S2P1','D3S2P2','D3S2P3','D3S3P1','D3S3P2','D3S3P3',
     'D4S1P1', 'D4S1P2', 'D4S1P3', 'D4S2P1','D4S2P2','D4S2P3','D4S3P1','D4S3P2','D4S3P3',
     'D5S1P1', 'D5S1P2', 'D5S1P3', 'D5S2P1','D5S2P2','D5S2P3','D5S3P1','D5S3P2','D5S3P3',
     'D6S1P1', 'D6S1P2', 'D6S1P3', 'D6S2P1','D6S2P2','D6S2P3','D6S3P1','D6S3P2','D6S3P3',
     'D7S1P1', 'D7S1P2', 'D7S1P3', 'D7S2P1','D7S2P2','D7S2P3','D7S3P1','D7S3P2','D7S3P3'
    ]

In [4]:
# Timeslot durations: Period 1 has 140 mins, Period 2 has 160 mins, Period 3 has 140 mins
A = {'D1S1P1': 140, 'D1S1P2': 160, 'D1S1P3': 140,
     'D1S2P1': 140, 'D1S2P2': 160, 'D1S2P3': 140,
     'D1S3P1': 140, 'D1S3P2': 160, 'D1S3P3': 140,
     'D2S1P1': 140, 'D2S1P2': 160, 'D2S1P3': 140,
     'D2S2P1': 140, 'D2S2P2': 160, 'D2S2P3': 140,
     'D2S3P1': 140, 'D2S3P2': 160, 'D2S3P3': 140,
     'D3S1P1': 140, 'D3S1P2': 160, 'D3S1P3': 140,
     'D3S2P1': 140, 'D3S2P2': 160, 'D3S2P3': 140,
     'D3S3P1': 140, 'D3S3P2': 160, 'D3S3P3': 140,
     'D4S1P1': 140, 'D4S1P2': 160, 'D4S1P3': 140,
     'D4S2P1': 140, 'D4S2P2': 160, 'D4S2P3': 140,
     'D4S3P1': 140, 'D4S3P2': 160, 'D4S3P3': 140,
     'D5S1P1': 140, 'D5S1P2': 160, 'D5S1P3': 140,
     'D5S2P1': 140, 'D5S2P2': 160, 'D5S2P3': 140,
     'D5S3P1': 140, 'D5S3P2': 160, 'D5S3P3': 140,
     'D6S1P1': 140, 'D6S1P2': 160, 'D6S1P3': 140,
     'D6S2P1': 140, 'D6S2P2': 160, 'D6S2P3': 140,
     'D6S3P1': 140, 'D6S3P2': 160, 'D6S3P3': 140,
     'D7S1P1': 140, 'D7S1P2': 160, 'D7S1P3': 140,
     'D7S2P1': 140, 'D7S2P2': 160, 'D7S2P3': 140,
     'D7S3P1': 140, 'D7S3P2': 160, 'D7S3P3': 140
    }

In [5]:
# Create the model
factory = gp.Model('Foam Planning')

# Add the variables

# If changeover happens at the end of timeslot k
y = factory.addVars(R, K, vtype=GRB.BINARY, name='y')

# If tool is placed on carrier in that time slot
x = factory.addVars(T, R, K, vtype=GRB.BINARY, name='x')

Using license file /Users/mayukh/gurobi.lic
Academic license - for non-commercial use only


In [6]:
# Add the constraints

# Total number of Tools on the carrier cannot be more than 
# the total number of available tools
for k in K:
    for i in T:
        factory.addConstr(gp.quicksum(x[i,j,k] for j in R) <= Q[i])

In [7]:
# The number of tools on a carrier is less than or equal to 1
# Complexity of matching and FBE will be added later
for k in K:
    for j in R:
        factory.addConstr(gp.quicksum(x[i,j,k] for i in T) <= 1)

In [8]:
# For each product (tool), the production should 
# be greater than or equal to the customer demand and the end inventory
req = factory.addConstrs((
    B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) >=
    D[i] + I[i] for i in T), name='req')

In [9]:
# Changeover happens only when the tool on a carrier is different 
# on a timeslot than the prior timeslot
diff = factory.addVars(T, R, K, vtype=GRB.INTEGER, lb=-2, ub=2, name='diff')
abs_diff = factory.addVars(T, R, K, vtype=GRB.INTEGER,  lb=-2, ub=2, name='abs_diff')

for j in R:
    for k in K:
        # The last one will always be 0
        if(k!=K[-1]):
            for i in T:
                factory.addConstr(diff[i, j, k] == x[i,j,k] - x[i,j,K[K.index(k)+1]])
                factory.addConstr(abs_diff[i, j, k] == abs_(diff[i, j, k]))
            
            factory.addConstr((y[j,k]==1) >> (gp.quicksum(abs_diff[i,j,k] for i in T)>=1))
            factory.addConstr((y[j,k]==0) >> (gp.quicksum(abs_diff[i,j,k] for i in T)<=0))

In [10]:
factory.modelSense = GRB.MINIMIZE

### Objective Scenario 1
- Minimize the number of changeovers has the highest priority
- Minimize production to ensure least on hand quantity above required

In [11]:
idx = 0

# Add the objective for Number of changeovers with a higher priority
obj2 = gp.quicksum(y[j,k] for j in R for k in K)
factory.setObjectiveN(obj2, idx, priority=2, name='O2')
idx += 1

# Add the objective for production with a lower priority
for i in T:
    obj = B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) - (D[i] + I[i])
    factory.setObjectiveN(obj, index=idx, priority=1, name='OBJCT'+str(idx))
    idx += 1

In [12]:
# Setting time limit
factory.Params.TimeLimit = 240*60
factory.update()

Changed value of parameter TimeLimit to 14400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [13]:
factory.write('Sc1-model.lp')

In [14]:
factory.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1811 rows, 4095 columns and 7500 nonzeros
Model fingerprint: 0x758ef67f
Model has 1860 general constraints
Variable types: 0 continuous, 4095 integer (1575 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 8e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 8118 rows and 7130 columns
Presolve time: 0.09s
Presolved: 9929 rows and 11225 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (O2) ...
------------------

  1838   711   24.00000   25  214 1200.00000   24.00000  98.0%   564  309s
  1917   721  500.00000   33  165 1200.00000   24.00000  98.0%   569  315s
  2094   680   24.00000   19  204 1200.00000   24.00000  98.0%   534  321s
  2213   638   24.00000   28  252 1200.00000   24.00000  98.0%   532  327s
  2318   596 infeasible   21      1200.00000   24.00000  98.0%   539  333s
  2398   583   24.00000   21  335 1200.00000   24.00000  98.0%   542  339s
  2529   545  558.19454   83  192 1200.00000   24.00000  98.0%   529  344s
  2622   530 infeasible   21      1200.00000   24.00000  98.0%   523  349s
  2728   492 infeasible   22      1200.00000   24.00000  98.0%   518  354s
  2801   467  239.36941   36   94 1200.00000   24.00000  98.0%   524  360s
  2883   447 infeasible   28      1200.00000   24.00000  98.0%   526  366s
  2950   461   24.00000   26  227 1200.00000   24.00000  98.0%   528  372s
  3027   512  948.00000   37   85 1200.00000   24.00000  98.0%   524  381s
  3260   477 infeasible  

In [15]:
# Export the plan to a .sol file
factory.write('Sc1-Sol.sol')

# Quantities of each Tool is produced
rows = K.copy()
columns = R.copy()
produced_qty = {'Tool1': 0, 'Tool2': 0, 'Tool3': 0, 'Tool4':0}
prod_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for i, j, k in x.keys():
    if(x[i, j, k].x==1):
        produced_qty[i] += L*A[k]
        prod_plan.loc[k, j]=i

print('Tool  ->\tProduced Qty\tCust Demand\tEnding Inv Req\tDifference')
for pq in produced_qty:
    print(pq, '->\t', produced_qty[pq], '\t', D[pq], '\t\t', I[pq], '\t\t', produced_qty[pq]-D[pq]-I[pq])
    
# Changeover by Carrier
rows = K.copy()
columns = R.copy()
change_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for j, k in y.keys():
    if(y[j,k].x==1):
        change_plan.loc[k, j]=y[j,k].x
print('Number of Changeovers by Carrier')
total_changeovers = 0
for j in R:
    print(j + '\t->\t' + str(change_plan[j].sum()))
    total_changeovers += change_plan[j].sum()
print('Total number of changeovers: '+ str(total_changeovers))

# Export the plan to a csv file
prod_plan.to_csv('Sc1-plan.csv')
change_plan.to_csv('Sc1-change.csv')

Tool  ->	Produced Qty	Cust Demand	Ending Inv Req	Difference
Tool1 ->	 8106.0 	 8000 		 50 		 56.0
Tool2 ->	 8120.0 	 8000 		 40 		 80.0
Tool3 ->	 9030.0 	 8000 		 30 		 1000.0
Tool4 ->	 7084.0 	 7000 		 20 		 64.0
Number of Changeovers by Carrier
C1	->	0.0
C2	->	1.0
C3	->	1.0
C4	->	1.0
C5	->	0.0
Total number of changeovers: 3.0


### Objective Scenario 2
- Minimize production to ensure least on hand quantity above required. Make this the highest priority
- Minimize the number of changeovers has lesser priority than the above

In [16]:
# Reset the objectives
factory.NumObj=0
factory.update()
factory.write('check.lp')

In [17]:
idx = 0

# Add the objective for production with a higher priority
for i in T:
    obj = B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) - (D[i] + I[i])
    factory.setObjectiveN(obj, index=idx, priority=2, name='OBJCT'+str(idx))
    idx += 1
    

# Add the objective for Number of changeovers with a lower priority
obj2 = gp.quicksum(y[j,k] for j in R for k in K)
factory.setObjectiveN(obj2, idx, priority=1, name='O2')
idx += 1

In [18]:
# Setting time limit
factory.Params.TimeLimit = 240*60
factory.update()

Parameter TimeLimit unchanged
   Value: 14400.0  Min: 0.0  Max: inf  Default: inf


In [19]:
factory.write('Sc2-model.lp')

In [20]:
factory.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1811 rows, 4095 columns and 7500 nonzeros
Model fingerprint: 0xfe0f5e96
Model has 1860 general constraints
Variable types: 0 continuous, 4095 integer (1575 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 8e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 8118 rows and 7130 columns
Presolve time: 0.05s
Presolved: 9929 rows and 11225 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
------------

  2731  1960    8.09091  241  189   33.00000    1.00000  97.0%   253  231s
  2971  2212    8.94805  288  173   33.00000    1.00000  97.0%   240  236s
  3457  2630    4.10811   73  182   33.00000    1.01187  96.9%   213  242s
  3710  2737    7.30526  148  130   33.00000    1.01187  96.9%   204  245s
  4028  3180   11.00000  249  114   33.00000    1.01187  96.9%   201  252s
  4289  3370   13.70800  304  169   33.00000    1.02273  96.9%   194  255s
  4655  3551    1.84247   55  268   33.00000    1.02273  96.9%   187  260s
  4852  3828    8.67424  167  142   33.00000    1.02273  96.9%   189  266s
  5022  3986    2.43297   33  268   33.00000    1.02273  96.9%   189  270s
  5553  4679   12.27273  206  230   33.00000    1.02273  96.9%   181  278s
  5999  4747    2.24009   30  281   33.00000    1.02400  96.9%   172  282s
  6067  4940    9.89600   42  132   33.00000    1.02400  96.9%   178  286s
  6263  5119   17.47469   77  234   33.00000    1.02400  96.9%   183  290s
  6673  5470   31.40909  

 28143 19533    9.26667  200  152   30.00000    1.46667  95.1%   168  868s
 28313 19748    8.42857  243  159   30.00000    1.46667  95.1%   168  871s
 28598 19826   25.00866  299  164   30.00000    1.46667  95.1%   167  875s
 28975 20116   26.14286  388   32   30.00000    1.50000  95.0%   167  883s
 29255 20202    8.00000   76  139   30.00000    1.50000  95.0%   166  886s
 29449 20304   22.13636  185  131   30.00000    1.51282  95.0%   166  890s
 29800 20600 infeasible  204        30.00000    1.53539  94.9%   166  898s
 30069 20700     cutoff  191        30.00000    1.54602  94.8%   166  902s
 30273 20831    5.19048  100  243   30.00000    1.54602  94.8%   165  905s
 30750 20947    3.08539   51  246   30.00000    1.54771  94.8%   164  912s
 30860 21123    5.49976   87  196   30.00000    1.54771  94.8%   164  916s
 31084 21196   27.20000  212  129   30.00000    1.55344  94.8%   164  920s
 31569 21479    5.42121  208  211   30.00000    1.55529  94.8%   163  928s
 31741 21572    2.55344  

 60498 41242    6.04598  226  217   29.00000    1.77505  93.9%   151 1417s
 60605 41477    4.11268   47  182   29.00000    1.77505  93.9%   151 1422s
 60864 41620    5.01463   92  153   29.00000    1.77505  93.9%   151 1426s
 61259 42016    5.14205  170  167   29.00000    1.77505  93.9%   151 1433s
 61433 42284    5.27273  227  116   29.00000    1.77505  93.9%   151 1436s
 61701 42609    7.27586  299  108   29.00000    1.77505  93.9%   151 1440s
 62396 43037    3.20443   49  287   29.00000    1.78243  93.9%   151 1446s
 62558 43281   20.38462  145  152   29.00000    1.78391  93.8%   151 1450s
 63040 43705   18.16216  140  165   29.00000    1.78700  93.8%   151 1456s
 63589 44157   19.16462  252   86   29.00000    1.79773  93.8%   150 1462s
 63778 44284    3.13572   58  239   29.00000    1.79773  93.8%   150 1465s
 64194 44837    7.54643  242  177   29.00000    1.79899  93.8%   150 1472s
 64523 45231    4.00000   80  166   29.00000    1.79899  93.8%   149 1476s
 65200 45872   23.27825  

 95649 73350    3.18788   41  261   29.00000    2.00000  93.1%   143 1966s
 95907 73631   12.13333  196  113   29.00000    2.00000  93.1%   143 1970s
 96202 73988    9.15822  124  141   29.00000    2.00000  93.1%   143 1975s
 96921 74590    7.71618   89  159   29.00000    2.00000  93.1%   142 1983s
 97277 74934   11.00000  113  124   29.00000    2.00000  93.1%   142 1988s
 97671 75187   10.06061   96  250   29.00000    2.00000  93.1%   142 1992s
 97950 75429    2.42803   48  275   29.00000    2.00000  93.1%   142 1996s
 98208 75780    5.24686  144  194   29.00000    2.00000  93.1%   142 2000s
 98588 76009    2.00659   41  198   29.00000    2.00000  93.1%   141 2005s
 99200 76752    5.55879  244  234   29.00000    2.00000  93.1%   141 2013s
 99627 76935 infeasible  490        29.00000    2.00000  93.1%   141 2018s
 99832 77225    3.64484  100  280   29.00000    2.00000  93.1%   141 2023s
 100148 77544    7.40000  266  117   29.00000    2.00000  93.1%   141 2028s
 100495 77796    3.32251

...

 906324 805864    5.40000  233   86   29.00000    2.14878  92.6%   117 12116s
 906790 805974    5.09692   61  162   29.00000    2.14879  92.6%   117 12120s
 907015 806268   25.12121  178   73   29.00000    2.14886  92.6%   117 12125s
 907506 806797    4.03030   69  273   29.00000    2.14890  92.6%   117 12131s
 907934 807072    4.42029   71  212   29.00000    2.14894  92.6%   117 12136s
 908158 807320    5.07576  126  159   29.00000    2.14894  92.6%   117 12140s
 908470 807691    5.22291  185   99   29.00000    2.14894  92.6%   117 12145s
 908926 808137    9.00000  274  131   29.00000    2.14894  92.6%   117 12150s
 909404 808576   25.16216  382   87   29.00000    2.14898  92.6%   117 12155s
 910221 809292    9.28571  290   58   29.00000    2.14907  92.6%   117 12162s
 910679 809628   12.67660   83  239   29.00000    2.14907  92.6%   117 12166s
 911082 809967   19.00000  105  134   29.00000    2.14907  92.6%   117 12171s
 911424 810312    4.93939   93  177   29.00000    2.14909  92.6%

 953125 847869   16.83562  158   97   29.00000    2.15385  92.6%   116 12646s
 953389 848130    5.36282   81  167   29.00000    2.15385  92.6%   116 12651s
 953694 848399    8.10744  140  121   29.00000    2.15385  92.6%   116 12655s
 953993 848731    6.46118  220  134   29.00000    2.15385  92.6%   116 12660s
 954612 849222    8.05085  102  129   29.00000    2.15385  92.6%   116 12666s
 954910 849563    3.73191   51  261   29.00000    2.15385  92.6%   116 12671s
 955371 849933    8.15544  263  119   29.00000    2.15385  92.6%   116 12675s
 955638 850182   12.40678  107  167   29.00000    2.15385  92.6%   116 12680s
 955983 850516    5.33546   92  497   29.00000    2.15385  92.6%   116 12686s
 956351 850845    7.75357  176  248   29.00000    2.15385  92.6%   116 12691s
 956700 851152    7.00438   86  350   29.00000    2.15385  92.6%   116 12695s
 957152 851526 infeasible  302        29.00000    2.15385  92.6%   116 12700s
 957480 851920   13.00000  143   90   29.00000    2.15388  92.6%

 995839 886224    6.18767   63  237   29.00000    2.15909  92.6%   116 13176s
 996056 886459   21.21134  173  109   29.00000    2.15909  92.6%   116 13180s
 996418 886749    3.68366  175  180   29.00000    2.15909  92.6%   116 13186s
 996701 886985    3.19756   58  319   29.00000    2.15909  92.6%   116 13191s
 996930 887321    5.00192  183  180   29.00000    2.15909  92.6%   116 13196s
 997491 887724    2.50735   73  230   29.00000    2.15909  92.6%   116 13200s
 997847 888205    5.15661  189  179   29.00000    2.15909  92.6%   116 13205s
 998464 888736    4.09095  147  119   29.00000    2.15909  92.6%   116 13210s
 998937 889112    3.58223  145  136   29.00000    2.15976  92.6%   116 13215s
 999339 889447    6.00000   60  178   29.00000    2.15977  92.6%   116 13220s
 999897 889907   17.12121  160  125   29.00000    2.15993  92.6%   116 13227s
 1000269 890298    6.25061  220  130   29.00000    2.16009  92.6%   116 13231s
 1000776 890668    5.67016  141  123   29.00000    2.16009  92.

 1038675 924437     cutoff  279        29.00000    2.16414  92.5%   116 13695s
 1038999 924794   12.82572  108  336   29.00000    2.16414  92.5%   116 13700s
 1039420 925122    4.21858   62  162   29.00000    2.16414  92.5%   116 13705s
 1039840 925448    3.06596   82  236   29.00000    2.16414  92.5%   116 13710s
 1040159 925789    7.22750  202  123   29.00000    2.16414  92.5%   116 13715s
 1040619 926187    3.73749  126  250   29.00000    2.16417  92.5%   116 13722s
 1040954 926597    5.95559  202  456   29.00000    2.16417  92.5%   116 13727s
 1041298 926761   12.51210  292   90   29.00000    2.16419  92.5%   116 13730s
 1041743 927183    4.44513   64  101   29.00000    2.16430  92.5%   116 13736s
 1042287 927607    6.38835   73  129   29.00000    2.16433  92.5%   116 13741s
 1042716 928028    4.62462  125  142   29.00000    2.16438  92.5%   116 13746s
 1043145 928357    6.00000   94  151   29.00000    2.16438  92.5%   116 13751s
 1043557 928888    8.85714  316  130   29.00000    2

 1079972 961229    5.09091  227   85   29.00000    2.16705  92.5%   116 14217s
 1080408 961790    9.85617  133  155   29.00000    2.16712  92.5%   116 14221s
 1081007 962030    8.99737   83  296   29.00000    2.16721  92.5%   116 14226s
 1081298 962397   28.00000  250   86   29.00000    2.16730  92.5%   116 14231s
 1081851 962920    5.20000  216  153   29.00000    2.16737  92.5%   116 14236s
 1082192 963204   20.00000  132   84   29.00000    2.16756  92.5%   116 14240s
 1082619 963550   21.63636  369   85   29.00000    2.16757  92.5%   116 14245s
 1082898 963774    7.23066  101  215   29.00000    2.16757  92.5%   116 14251s
 1083129 964226    9.26471  213   92   29.00000    2.16757  92.5%   116 14256s
 1083664 964607    9.73786  100  132   29.00000    2.16808  92.5%   116 14260s
 1084187 964921    4.27291   63  339   29.00000    2.16818  92.5%   116 14265s
 1084487 965321   16.00000  144  101   29.00000    2.16818  92.5%   116 14270s
 1084935 965556    8.57331   64  323   29.00000    2

In [21]:
# Export the plan to a .sol file
factory.write('Sc2-Sol.sol')

# Quantities of each Tool is produced
rows = K.copy()
columns = R.copy()
produced_qty = {'Tool1': 0, 'Tool2': 0, 'Tool3': 0, 'Tool4':0}
prod_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for i, j, k in x.keys():
    if(x[i, j, k].x==1):
        produced_qty[i] += L*A[k]
        prod_plan.loc[k, j]=i

print('Tool  ->\tProduced Qty\tCust Demand\tEnding Inv Req\tDifference')
for pq in produced_qty:
    print(pq, '->\t', produced_qty[pq], '\t', D[pq], '\t\t', I[pq], '\t\t', produced_qty[pq]-D[pq]-I[pq])
    
# Changeover by Carrier
rows = K.copy()
columns = R.copy()
change_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for j, k in y.keys():
    if(y[j,k].x==1):
        change_plan.loc[k, j]=y[j,k].x
print('Number of Changeovers by Carrier')
total_changeovers = 0
for j in R:
    print(j + '\t->\t' + str(change_plan[j].sum()))
    total_changeovers += change_plan[j].sum()
print('Total number of changeovers: '+ str(total_changeovers))

# Export the plan to a csv file
prod_plan.to_csv('Sc2-plan.csv')
change_plan.to_csv('Sc2-change.csv')

Tool  ->	Produced Qty	Cust Demand	Ending Inv Req	Difference
Tool1 ->	 8050.0 	 8000 		 50 		 0.0
Tool2 ->	 8050.0 	 8000 		 40 		 10.0
Tool3 ->	 8036.0 	 8000 		 30 		 6.0
Tool4 ->	 7028.0 	 7000 		 20 		 8.0
Number of Changeovers by Carrier
C1	->	4.0
C2	->	4.0
C3	->	3.0
C4	->	7.0
C5	->	11.0
Total number of changeovers: 29.0


### Objective Scenario 3
- Minimize production to ensure least on hand quantity above required
- Minimize the number of changeovers
- No Prioritization specified for either of these objectives

In [22]:
# Reset the objectives
factory.NumObj=0
factory.update()
factory.write('check.lp')

In [23]:
idx = 0

# Add the objective for Number of changeovers with a lower priority
obj2 = gp.quicksum(y[j,k] for j in R for k in K)
factory.setObjectiveN(obj2, idx, name='O2')
idx += 1

# Add the objective for production with a higher priority
for i in T:
    obj = B[i] + L*(gp.quicksum(A[k]*(gp.quicksum(x[i,j,k] for j in R)) for k in K)) - (D[i] + I[i])
    factory.setObjectiveN(obj, index=idx, name='OBJCT'+str(idx))
    idx += 1

In [24]:
# Setting time limit
factory.Params.TimeLimit = 240*60
factory.update()

Parameter TimeLimit unchanged
   Value: 14400.0  Min: 0.0  Max: inf  Default: inf


In [25]:
factory.write('Sc3-model.lp')

In [26]:
factory.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1811 rows, 4095 columns and 7500 nonzeros
Model fingerprint: 0x051cc4c4
Model has 1860 general constraints
Variable types: 0 continuous, 4095 integer (1575 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 8e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 1811 rows, 4095 columns and 7500 nonzeros
Model has 1860 general constraints
Variable types: 0 continuous, 4095 integer (1575 binary)
Coeffic

 14645 10983   34.83398  335  907   50.00000   25.09302  49.8%   197  339s
 14646 10984   43.04889  276  907   50.00000   25.09302  49.8%   197  341s
 14648 10988   26.84653   19  784   50.00000   25.09302  49.8%   200  377s
 14650 10990   30.53409   20  903   50.00000   25.09302  49.8%   200  403s
 14652 10991   27.86634   20  790   50.00000   25.09302  49.8%   201  438s
 14654 10992   27.88366   21  792   50.00000   25.09302  49.8%   201  444s
 14656 10994   28.36472   21  603   50.00000   25.09302  49.8%   201  449s
 14658 10995   28.84653   22  539   50.00000   25.09302  49.8%   201  455s
 14660 10996   30.21429   22  712   50.00000   25.09302  49.8%   202  464s
 14662 10999   28.84653   23  491   50.00000   25.09302  49.8%   202  467s
 14665 11001   30.00000   24  360   50.00000   25.09302  49.8%   202  472s
 14668 11004   30.00000   25  329   50.00000   25.09302  49.8%   202  476s
 14672 11006   30.33911   26  294   50.00000   25.09302  49.8%   203  482s
 14681 11013   31.00000  

 35546 22521   28.45836   65  119   50.00000   25.48084  49.0%   175  981s
 35704 22630   29.15385  103   62   50.00000   25.48084  49.0%   176  987s
 35782 22686   29.36364  127   58   50.00000   25.48084  49.0%   177  990s
 35954 22798   31.11059  181  104   50.00000   25.48084  49.0%   178  996s
 36076 22853   31.24408  223  182   50.00000   25.48084  49.0%   179 1000s
 36451 23024   34.71435  355  116   50.00000   25.48270  49.0%   179 1009s
 36713 23167   32.62267   67  155   50.00000   25.48518  49.0%   179 1013s
 36976 23272   27.75497   31  370   50.00000   25.48518  49.0%   179 1017s
 37193 23325   33.41551  105  549   50.00000   25.48518  49.0%   179 1021s
 37319 23515   39.66976  181  298   50.00000   25.48560  49.0%   179 1025s
 37729 23950   33.75599  153  235   50.00000   25.48604  49.0%   178 1034s
 38185 23982   26.41118   39  349   50.00000   25.48604  49.0%   177 1038s
 38391 24169   31.90897  160  354   50.00000   25.48604  49.0%   176 1043s
 38651 24158   44.69634  

 64685 44930   34.75338  117  350   50.00000   25.80343  48.4%   161 1531s
 65284 45431   28.61197   94  294   50.00000   25.81545  48.4%   160 1538s
 65533 45783   35.11729  254   91   50.00000   25.82080  48.4%   160 1542s
 65909 45917   28.94122   43  355   50.00000   25.82080  48.4%   159 1545s
 66306 46352   40.15992  203  328   50.00000   25.83927  48.3%   159 1553s
 66547 46451   26.33968   25  259   50.00000   25.83927  48.3%   159 1556s
 66789 46736   37.10811  101   69   50.00000   25.83927  48.3%   159 1562s
 66969 46771   49.00000  178   84   50.00000   25.83959  48.3%   159 1565s
 67338 46954   27.82080  109  192   50.00000   25.83959  48.3%   160 1572s
 67549 47125   28.75327  161  110   50.00000   25.83959  48.3%   159 1575s
 67863 47332   30.37802  253  100   50.00000   25.83959  48.3%   160 1582s
 67943 47473   31.15725  294   90   50.00000   25.85446  48.3%   160 1585s
 68243 47685   43.41306  119  202   50.00000   25.86598  48.3%   160 1591s
 68454 48014   31.95868  

 95813 72450   39.00000   92   83   50.00000   26.00000  48.0%   151 2087s
 95940 72566   43.10811  112   85   50.00000   26.00000  48.0%   152 2091s
 96088 72744   27.63266   49  297   50.00000   26.00000  48.0%   152 2095s
 96536 73170   28.01089   45  311   50.00000   26.00000  48.0%   152 2103s
 96777 73479   32.31921   95  194   50.00000   26.00000  48.0%   152 2108s
 97118 73599   27.35224   37  234   50.00000   26.00000  48.0%   152 2111s
 97238 73772   37.25766  110  157   50.00000   26.00000  48.0%   152 2116s
 97423 73918   30.27016   38  423   50.00000   26.00000  48.0%   152 2120s
 97590 74089   38.46959   94  294   50.00000   26.00000  48.0%   152 2125s
 97890 74370   35.34868   79  270   50.00000   26.00000  48.0%   152 2132s
 98065 74633   42.62252  205  138   50.00000   26.00000  48.0%   152 2136s
 98344 74779   28.17532   27  498   50.00000   26.00000  48.0%   152 2140s
 98508 75070   35.24540   65  673   50.00000   26.00000  48.0%   152 2145s
 98998 75327   30.97666  

...

 717148 611661   39.47187  193  134   49.00000   26.35783  46.2%   128 12238s
 717638 611874   29.01185   58  299   49.00000   26.35797  46.2%   128 12241s
 717851 612184   33.04855  183   96   49.00000   26.35808  46.2%   128 12245s
 718624 612686   27.18562   42  300   49.00000   26.35816  46.2%   128 12251s
 718893 612995   45.00000  170   73   49.00000   26.35842  46.2%   128 12257s
 719060 613145   30.39437   60  125   49.00000   26.35865  46.2%   128 12261s
 719388 613541   28.28111   80  135   49.00000   26.35877  46.2%   128 12267s
 719656 613682   37.27586  239   66   49.00000   26.35883  46.2%   128 12270s
 719948 613968   41.17266  120  117   49.00000   26.35884  46.2%   128 12276s
 720301 614406   38.26813  134  116   49.00000   26.35907  46.2%   128 12283s
 720595 614648   26.59034   41  248   49.00000   26.35907  46.2%   128 12286s
 721002 614911   38.04695   75  140   49.00000   26.35907  46.2%   128 12293s
 721152 615186   44.21405   96  134   49.00000   26.35916  46.2%

 750023 641284   28.79517  102  216   49.00000   26.36892  46.2%   128 12765s
 750424 641663   29.52756   76  225   49.00000   26.36900  46.2%   128 12772s
 750631 641998   40.07243  190  224   49.00000   26.36909  46.2%   128 12776s
 751016 642285   29.15346   51  424   49.00000   26.36909  46.2%   128 12780s
 751683 642741   29.17501   57  272   49.00000   26.36928  46.2%   128 12787s
 751822 642924   35.05789  100  138   49.00000   26.36935  46.2%   128 12790s
 752178 643224   38.75714  122  300   49.00000   26.36948  46.2%   128 12797s
 752381 643521   30.06198   53  162   49.00000   26.36966  46.2%   128 12801s
 752678 643946   33.63272  179  128   49.00000   26.36966  46.2%   128 12805s
 753672 644542   32.44511  353  113   49.00000   26.36967  46.2%   128 12812s
 753773 644624   27.20091   51  289   49.00000   26.36984  46.2%   128 12815s
 753855 644830   27.42214   62  223   49.00000   26.36984  46.2%   128 12820s
 754244 645213   30.03030   96  116   49.00000   26.36984  46.2%

 784976 672671   29.09752  101  188   49.00000   26.37894  46.2%   127 13295s
 785650 673258   30.16327  122   84   49.00000   26.37894  46.2%   127 13302s
 785941 673541   35.92278  340  103   49.00000   26.37895  46.2%   127 13306s
 786260 673807   32.86096  137  133   49.00000   26.37899  46.2%   127 13310s
 786682 674275   33.01774   79  122   49.00000   26.37899  46.2%   127 13316s
 787048 674413   47.48000  240   90   49.00000   26.37908  46.2%   127 13320s
 787472 674928   29.32054  250  194   49.00000   26.37913  46.2%   127 13326s
 787843 675175   29.04388   71  207   49.00000   26.37913  46.2%   127 13332s
 788011 675286   29.16385   82  255   49.00000   26.37913  46.2%   127 13335s
 788239 675488   29.56822  105  177   49.00000   26.37913  46.2%   127 13341s
 788649 676013   34.15045  191  145   49.00000   26.37913  46.2%   127 13348s
 788921 676093   34.28130  214   90   49.00000   26.37913  46.2%   127 13351s
 789119 676337   34.43401  264  177   49.00000   26.37913  46.2%

 816612 701327   31.50221  214  166   49.00000   26.38627  46.2%   127 13826s
 816896 701626   29.02740   62  181   49.00000   26.38627  46.2%   127 13830s
 817554 702253   29.01652   91  179   49.00000   26.38634  46.2%   127 13837s
 817874 702479   34.47544  307  263   49.00000   26.38644  46.2%   127 13841s
 818162 702634   30.77161   70  155   49.00000   26.38662  46.1%   127 13845s
 818546 703185   28.88179   95  214   49.00000   26.38671  46.1%   127 13852s
 818906 703390   31.46781  333  209   49.00000   26.38675  46.1%   127 13855s
 819239 703587   33.26182   70  156   49.00000   26.38675  46.1%   127 13861s
 819481 703780     cutoff  121        49.00000   26.38686  46.1%   127 13868s
 819744 703861   27.86505   46  511   49.00000   26.38686  46.1%   127 13871s
 819825 704073   29.70405   89  227   49.00000   26.38686  46.1%   127 13875s
 820206 704378   30.00788   62  184   49.00000   26.38725  46.1%   127 13882s
 820369 704694   29.65039   52  137   49.00000   26.38725  46.1%

 850881 732367   29.38991   59  251   49.00000   26.39434  46.1%   127 14357s
 851235 732544   37.11616  157  183   49.00000   26.39434  46.1%   127 14360s
 851605 732892   32.90737   86  161   49.00000   26.39455  46.1%   127 14367s
 851817 733308   28.00000   46   91   49.00000   26.39455  46.1%   127 14371s
 852234 733566   31.40000  253  197   49.00000   26.39465  46.1%   127 14375s
 852902 734274   32.25926  106  123   49.00000   26.39497  46.1%   127 14382s
 853328 734484   38.69412  235   89   49.00000   26.39509  46.1%   127 14386s
 853707 734854   30.48897  100   96   49.00000   26.39509  46.1%   127 14392s
 853957 735123   32.48157  214  170   49.00000   26.39509  46.1%   127 14395s
 854366 735287   31.08876  125  146   49.00000   26.39523  46.1%   127 14400s

Cutting planes:
  Gomory: 1
  MIR: 1
  Zero half: 1
  Mod-K: 15

Explored 854408 nodes (108511080 simplex iterations) in 14400.01 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 49 50 50 ... 7

In [27]:
# Export the plan to a .sol file
factory.write('Sc3-Sol.sol')

# Quantities of each Tool is produced
rows = K.copy()
columns = R.copy()
produced_qty = {'Tool1': 0, 'Tool2': 0, 'Tool3': 0, 'Tool4':0}
prod_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for i, j, k in x.keys():
    if(x[i, j, k].x==1):
        produced_qty[i] += L*A[k]
        prod_plan.loc[k, j]=i

print('Tool  ->\tProduced Qty\tCust Demand\tEnding Inv Req\tDifference')
for pq in produced_qty:
    print(pq, '->\t', produced_qty[pq], '\t', D[pq], '\t\t', I[pq], '\t\t', produced_qty[pq]-D[pq]-I[pq])
    
# Changeover by Carrier
rows = K.copy()
columns = R.copy()
change_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
for j, k in y.keys():
    if(y[j,k].x==1):
        change_plan.loc[k, j]=y[j,k].x
print('Number of Changeovers by Carrier')
total_changeovers = 0
for j in R:
    print(j + '\t->\t' + str(change_plan[j].sum()))
    total_changeovers += change_plan[j].sum()
print('Total number of changeovers: '+ str(total_changeovers))

# Export the plan to a csv file
prod_plan.to_csv('Sc3-plan.csv')
change_plan.to_csv('Sc3-change.csv')

Tool  ->	Produced Qty	Cust Demand	Ending Inv Req	Difference
Tool1 ->	 8050.0 	 8000 		 50 		 0.0
Tool2 ->	 8050.0 	 8000 		 40 		 10.0
Tool3 ->	 8036.0 	 8000 		 30 		 6.0
Tool4 ->	 7028.0 	 7000 		 20 		 8.0
Number of Changeovers by Carrier
C1	->	4.0
C2	->	2.0
C3	->	7.0
C4	->	9.0
C5	->	3.0
Total number of changeovers: 25.0


Thank you